<a href="https://colab.research.google.com/github/DennisMoYung/hk_parkingmeter/blob/main/hk_parking_meter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

In [91]:
# @title install and import

# install pandasql
!pip install pandasql



# import urllib library
#from urllib.request import Request, urlopen
import urllib.parse

#widget library
# import ipywidgets as widgets

# import json
import json

#import pandas
import pandas as pd
from pandasql import sqldf

#import numpy
import numpy as np

#import folium (map api)
import folium

In [92]:
# @title function

#generate PopUp Content for the map
def generatePopUpContent(id, row):
  result = ""
  #result = result +  "<h6>" + row['ParkingSpaceId'] + "</h6>"
  result = result + "<table class='table'>"
  result = result + "<tr><th>Parking Space Id: </th><td>"+ id +"</td></tr>"
  result = result + "<tr style='background-color: rgb(240,240,240);'><th>Parking Period Unit: </th><td>" + "$" + str(int(row['PaymentUnit'])) + " / " +str(int(row['TimeUnit'])) + "  min." + "</td></tr>"
  result = result + "<tr><th>Operating period: </th><td>"

  if ('A' in row['OperatingPeriod']):
    result = result + '08.00 am - Midnight on Mondays to Saturdays (except Sundays and public holidays)'
  elif ('B' in row['OperatingPeriod']):
    result = result + '08.00 am - 08.00 pm daily on Mondays to Saturdays (except Sundays and public holidays)'
  elif ('D' in row['OperatingPeriod']):
    result = result + '08.00 am - Midnight on Mondays to Saturdays;<br>10.00 am - 10.00 pm on Sundays and public holidays'
  elif ('E' in row['OperatingPeriod']):
    result = result + '07.00 am - 08.00 pm daily'
  elif ('F' in row['OperatingPeriod']):
    result = result + '08.00 am - 09.00 pm daily'
  elif ('H' in row['OperatingPeriod']):
    result = result + '08.00 am - 08.00 pm daily'
  elif ('J' in row['OperatingPeriod']):
    result = result + '08.00 am - Midnight daily'
  elif ('N' in row['OperatingPeriod']):
    result = result + '07.00 pm - Midnight daily'
  elif ('P' in row['OperatingPeriod']):
    result = result + '08.00 am - 08.00 pm daily on Mondays to Saturdays (no parking on Sundays)'
  elif ('Q' in row['OperatingPeriod']):
    result = result + '08.00 am - 08.00 pm daily on Mondays to Saturdays;<br>10.00 am - 10.00 pm daily on Sundays and public holidays'
  else:
    result = result + 'No parking on 08.00 am - 05.00 pm daily on Mondays to Fridays;<br>05.00 pm - Midnight daily on Mondays to Fridays;<br>08.00 am - Midnight daily on Saturdays;<br>10.00 am - 10.00 pm daily on Sundays and public holidays'

  result = result + "</td></tr>"
  result = result + "<tr style='background-color: rgb(240,240,240);'><th>Longest Parking Period: </th><td>" + str(int(row['LPP'])) + " min. (Each) </td><tr>"

  #status of meter
  if row['ParkingMeterStatus'] == 'NU':
    result = result +  "<tr><th>Status: </th><td> Not Available </td></tr>"
  elif row['OccupancyStatus'] == 'V':
    result = result +  "<tr><th>Status: </th><td>  Vacant </td></tr>"
  else:
    result = result +  "<tr><th>Status: </th><td> Occupied </td></tr>"

  result = result + "</table>"

  return result

#generate API query
def generateQuery(resource_url, section, filter, dev_mode = False):

  if filter is None:
    api_query = {"resource" :resource_url, "section" : section, "format" : "csv" }
  else:
    api_query = {"resource" :resource_url, "section" : section, "format" : "csv", "filters" : filter }

  api_query = json.dumps(api_query)
  api_query = api_query.replace(": ", ":").replace(", ", ",")

  if dev_mode:
    print(api_query)
    print()

  api_query = urllib.parse.quote(api_query)

  if(dev_mode):
    print(api_query)
    print()

  return api_query


# Data Control

In [93]:
# @title source URL and API

# URL for Distribution of Metered Parking Spaces
url_distribution = 'https://resource.data.one.gov.hk/td/psiparkingspaces/spaceinfo/parkingspaces.csv'

# URL for Occupancy Status of Metered Parking Spaces Installed with New Parking Meters
url_usage = 'https://resource.data.one.gov.hk/td/psiparkingspaces/occupancystatus/occupancystatus.csv'

# URL for Gov. API
url_gov_api = 'https://api.data.gov.hk/v2/filter'

In [94]:
# @title local Variables (include dev. mode) {display-mode: "form"}

dev_mode = False # @param {"type":"boolean","placeholder":"development Mode"}


In [95]:
# @title location filters { display-mode: "form" }

if dev_mode == True:

  # query for all data
  api_query_dev = generateQuery(url_distribution, 1, None, dev_mode)


  #form the URL for received all data
  url_query_dev = url_gov_api + "?q=" + api_query_dev

  #load into pandas (dev. mode)
  distribution = pd.read_csv(url_query_dev)
  #distribution.head()

  #create district list
  dist = distribution['District_tc'].unique()
  dist =  dist[~pd.isnull(dist)]
  dist_en = distribution['District'].unique()

  dist_arr = np.array(dist_en[:] + ", " + dist[:], dtype=object)
  dist_arr = np.array2string(dist_arr, separator=",")
  dist_arr = dist_arr.replace("'", "\"")

  print(dist_arr)
  print()

  tmp = sqldf("SELECT DISTINCT(District) as district FROM distribution WHERE Region LIKE 'NEW TERRITORIES';")
  tmp = tmp['district'].to_list()
  print(tmp)
  print()

# end of dev mode
############################


# region
region = "KOWLOON, 九龍" # @param ["HONG KONG, 香港島","KOWLOON, 九龍","NEW TERRITORIES, 新界"]
region = region.split(", ")

# district
district = "KOWLOON CITY, 九龍城" # @param ["SOUTHERN, 南區","SHAM SHUI PO, 深水埗","WAN CHAI, 灣仔","SHA TIN, 沙田", "WONG TAI SIN, 黃大仙","KWUN TONG, 觀塘","KOWLOON CITY, 九龍城","EASTERN, 東區", "CENTRAL & WESTERN, 中西區","TSUEN WAN, 荃灣","KWAI TSING, 葵青","ISLANDS, 離島", "TUEN MUN, 屯門","YUEN LONG, 元朗","NORTHERN, 北區","TAI PO, 大埔","SAI KUNG, 西貢", "YAU TSIM MONG, 油尖旺","INTERNAL TEST, 內部測試"]
district = district.split(", ")

# District Exception
kln_district = ['SHAM SHUI PO', 'WONG TAI SIN', 'KWUN TONG', 'KOWLOON CITY', 'YAU TSIM MONG', 'INTERNAL TEST']
hongkong_island_district = ['SOUTHERN', 'WAN CHAI', 'EASTERN', 'CENTRAL & WESTERN', 'INTERNAL TEST']
nt_district = ['SHA TIN', 'TSUEN WAN', 'KWAI TSING', 'ISLANDS', 'TUEN MUN', 'YUEN LONG', 'NORTHERN', 'TAI PO', 'SAI KUNG', 'INTERNAL TEST']

if (region[0] == "HONG KONG" and district[0] not in hongkong_island_district ) or (region[0] == "KOWLOON" and district[0] not in kln_district ) or (region[0] == "NEW TERRITORIES" and district[0] not in nt_district ) :
  raise Exception("Please select district under " + region[0])


################################

#Chinese and English are also available
#Please enter the full name

# specific location
specific_loc = "忠義街" # @param {"type":"string","placeholder":"Please enter the key and use comma to split them, [Example: 大網仔路, Pak Tam Chung Car Park ...]"}
specific_loc = specific_loc.upper()


################################

# Vehicle Type
vehicle = "Any Vehicles, 私家車/ 輕型貨車/ 小型巴士 (other than Medium and Heavy Goods Vehicles, Buses, Motor Cycles and Pedal Cycles)" # @param ["Coaches, 長途汽車","Goods Vehicles, 貨車","Any Vehicles, 私家車/ 輕型貨車/ 小型巴士 (other than Medium and Heavy Goods Vehicles, Buses, Motor Cycles and Pedal Cycles)"]
vehicle = vehicle[0]



In [96]:
# @title loading data

#gerneral info
# query with filters
filter = [[3, "eq", [region[0]]], [6, "eq", [district[0]]], [20, "eq", [vehicle]]]
api_query = generateQuery(url_distribution, 1, filter, dev_mode)

url_query = url_gov_api + "?q=" + api_query

#load into pandas
distribution = pd.read_csv(url_query)

#form sql condition
sql_query = ""

#with specific loc
if specific_loc != "":

  specific_loc_arr = specific_loc.split(",")
  for idx in range(0, len(specific_loc_arr)):
    specific_loc_arr[idx] = "'%" + specific_loc_arr[idx].strip() + "%'"
    sql_query = sql_query + "Street LIKE " + specific_loc_arr[idx] + " OR Street_tc LIKE " + specific_loc_arr[idx] + " OR SectionOfStreet LIKE " + specific_loc_arr[idx] + " OR SectionOfStreet_tc LIKE " + specific_loc_arr[idx] + " "
    if idx < len(specific_loc_arr) - 1:
      sql_query = sql_query + "OR "

  sql_query = "SELECT * FROM distribution WHERE " + sql_query + ";"

else:

  sql_query = "SELECT * FROM distribution;"

distribution = sqldf(sql_query)

# print(sqldf(sql_query))

# load usage of meter
usage = pd.read_csv(url_usage, storage_options={'User-Agent': 'Mozilla/5.0'}).set_index('ParkingSpaceId')
distribution = distribution.set_index('ParkingSpaceId').join(usage)

# distribution.head()



# Map rendering


In [97]:
if len(distribution.index) == 0:
  map_render = folium.Map(location=[22.28416553, 114.152666056],  zoom_start=12, tiles='OpenStreetMap')
else:
  map_render = folium.Map(location=[distribution['Latitude'].mean(), distribution['Longitude'].mean()],  zoom_start=16, tiles='OpenStreetMap')

#Group for meter that is not available
group_notUse = folium.FeatureGroup("Meters Not Available").add_to(map_render)
group_vacant = folium.FeatureGroup("Vancant").add_to(map_render)
group_occupied = folium.FeatureGroup("Occupied").add_to(map_render)

for idx, row in distribution.iterrows():
  #print(row['Latitude'], row['Longitude'] )
  # folium.Marker(location=[row['Latitude'], row['Longitude']], popup= folium.Popup(generatePopUpContent(idx, row), max_width=700)).add_to(group_notUse)
  if row['ParkingMeterStatus'] == 'NU':
    folium.Marker(location=[row['Latitude'], row['Longitude']], popup= folium.Popup(generatePopUpContent(idx, row), max_width=700), icon=folium.Icon("lightgray"), lazy=True).add_to(group_notUse)
  elif row['OccupancyStatus'] == 'V':
    folium.Marker(location=[row['Latitude'], row['Longitude']], popup= folium.Popup(generatePopUpContent(idx, row), max_width=700), icon=folium.Icon("green"), lazy=True).add_to(group_vacant)
  else:
    folium.Marker(location=[row['Latitude'], row['Longitude']], popup= folium.Popup(generatePopUpContent(idx, row), max_width=700), icon=folium.Icon("red"), lazy=True).add_to(group_occupied)

folium.LayerControl().add_to(map_render)

map_render


# **Reference**

*  *Website for Data.gov.hk*<br>
    [Data.gov.hk](https://data.gov.hk/en-data/dataset/hk-td-msd_1-metered-parking-spaces-data)

    <br>
*  *Link for datasheet:*<br>
    [Data Specification for Distribution of Metered Parking Spaces and Occupancy of those Installed with New Parking Meters](https://www.td.gov.hk/datagovhk_td/metered-parking-spaces-data/resources/tc/dataspec/metered_parking_spaces_data_dataspec.pdf) <br><br>
    [data description from data.gov.hk](https://static.csdi.gov.hk/csdi-webpage/view/946d6d8b3baa52d3bce6b0279c1e1b24/sds_view)

    <br>
*  *API interface:*<br>
    [API Specification](https://data.gov.hk/en/help/api-spec#api-for-data-filtering)
